<a href="https://colab.research.google.com/github/webb-e/S2_Landsat_Comparison/blob/main/Batch_LakeOccurrence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
author: @ericslevenson, updated by @ElizabethWebb
date: June 2024
description: Ingest S2 tiles, export weekly lake occurance map
'''

'\nauthor: @ericslevenson, updated by @ElizabethWebb\ndate: June 2024\ndescription: Ingest S2 tiles, export weekly lake occurance map\n'

In [ ]:
# Authenticate private account (only required for exporting to drive/gee/gcp)
from google.colab import auth
auth.authenticate_user()

# Earth Engine setup
import ee # Trigger the authentication flow.
ee.Authenticate()

# Initialize the library. ## add your project ID here
ee.Initialize(project=" ")

# Some common imports
from IPython.display import Image
import folium

In [ ]:
regionfolder = None
lakestring = None
ids = None
def regionfun(region):
  global regionfolder, lakestring, ids
  if region == 'TUK/MRD':
    regionfolder = 'TUK_MRD_weekly/'
    lakestring = "projects/alpod-412314/assets/Lake_extractions/TUK_MRD_AND_extraction"
    ids = ['07WFS', '07WFT', '08WMA', '08WMB', '08WMC', '08WMV', '08WNA', '08WNB', '08WNC', '08WNV', '08WPB', '08WPC', '08WPD', '09WVT', '09WVU']
  elif region == 'AKCP':
    regionfolder = 'AKCP_weekly/'
    lakestring = "projects/alpod-412314/assets/Lake_extractions/AKCP"
    ids = ['03WWT', '03WXT', '03WXU', '04WDC', '04WDD', '04WEC', '04WED', '04WEE', '04WFC', '04WFD', '04WFE', '05WMT', '05WMU', '05WMV', '05WNT',
         '05WNU', '05WPT','05WPU', '06WVC', '06WVD', '06WWC', '06WXC', '07WDT']
  elif region == 'YKD':
    regionfolder = 'YKD_weekly/'
    lakestring = "projects/alpod-412314/assets/Lake_extractions/YKD"
    ids = ['03VVH', '03VVJ', '03VVK', '03VWJ', '03VWG', '03VWH','03VWK', '03VWL', '03VXG', '03VXH', '03VXJ', '03VXK', '03VXL', '04VCM', '04VCN', '04VCP', '04VCQ',
          '04VCR', '04VDM', '04VDN', '04VDP', '04VEP']
  elif region == 'YKF':
    regionfolder = 'YKF_weekly/'
    lakestring = "projects/alpod-412314/assets/Lake_extractions/YKF"
    ids = ['05WPP', '05WPQ', '06WVU', '06WVV', '06WWT', '06WWU', '06WWV', '06WXU', '06WXV', '07WDP', '07WDQ']

  else:
    print("Invalid region")

In [ ]:
## *** INPUTS THAT NEED TO BE ADJUSTED ***
year = str(2016)
regionfun('AKCP') # options = TUK/MRD, AKCP, YKD, YKF
lakes = ee.FeatureCollection(lakestring)

start1 = year +'-05-01'
finish1 = year +'-09-30'

# Tiled ROI
tiles = ee.FeatureCollection('projects/ee-webbe/assets/ALPOD/S2_tiles')
pixScale = 10
eestart = ee.Date(start1)
eefinish = ee.Date(finish1)

### define weeks
n_weeks = eefinish.difference(eestart,'week').round();
dates = ee.List.sequence(0, n_weeks, 1);
def make_datelist(n):
      return eestart.advance(n,'week')

dates = dates.map(make_datelist)


dates_list = dates.getInfo()
print(dates_list)

[{'type': 'Date', 'value': 1462060800000}, {'type': 'Date', 'value': 1462665600000}, {'type': 'Date', 'value': 1463270400000}, {'type': 'Date', 'value': 1463875200000}, {'type': 'Date', 'value': 1464480000000}, {'type': 'Date', 'value': 1465084800000}, {'type': 'Date', 'value': 1465689600000}, {'type': 'Date', 'value': 1466294400000}, {'type': 'Date', 'value': 1466899200000}, {'type': 'Date', 'value': 1467504000000}, {'type': 'Date', 'value': 1468108800000}, {'type': 'Date', 'value': 1468713600000}, {'type': 'Date', 'value': 1469318400000}, {'type': 'Date', 'value': 1469923200000}, {'type': 'Date', 'value': 1470528000000}, {'type': 'Date', 'value': 1471132800000}, {'type': 'Date', 'value': 1471737600000}, {'type': 'Date', 'value': 1472342400000}, {'type': 'Date', 'value': 1472947200000}, {'type': 'Date', 'value': 1473552000000}, {'type': 'Date', 'value': 1474156800000}, {'type': 'Date', 'value': 1474761600000}, {'type': 'Date', 'value': 1475366400000}]


### Main function

In [ ]:
for i, id in enumerate(ids):
  for d1 in dates_list:
     # d1 = ee.Date(start).format('YYYY-MM-dd')
      d1 = d1['value']
      start = ee.Date(d1);
      end = ee.Date(d1).advance(1, 'week');
      date_range = ee.DateRange(start, end);
      eeroi = tiles.filter(ee.Filter.eq('Name', id)).first() # Filter grid to tile to define roi
      roi = ee.Geometry.Polygon(eeroi.geometry().getInfo()['coordinates'][0]) # define roi as geometry variable

       #### EXPORT DESCRIPTION
      year = start.get('year').getInfo();
      month = start.get('month').getInfo();
      week  = start.get('week').getInfo();

      description = str(id) + "_" + str(year) + "_" + str(week) # Export Name
      ###############################################################################
      ## ***IMAGE PRE-PROCESSING METHODS***
      ###############################################################################

      # get images and cloud probability. help from: https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless
      def get_s2_sr_cld_col(aoi):
          # Import and filter S2 SR.
          s2_sr_col = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterBounds(aoi).filterDate(start,end).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',50))
          # Import and filter s2cloudless
          s2_cloudless_col = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY').filterBounds(aoi).filterDate(start,end)
          # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
          return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
              'primary': s2_sr_col,
              'secondary': s2_cloudless_col,
              'condition': ee.Filter.equals( # Pass leftField and rightField as separate arguments
                  leftField= 'system:index',
                  rightField= 'system:index'
                  )
              }))

      def add_cloud_bands(img):
          # Get s2cloudless image, subset the probability band.
          cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

          # Condition s2cloudless by the probability threshold value.
          is_cloud = cld_prb.gt(50).rename('clouds')

          # Add the cloud probability layer and cloud mask as image bands.
          return img.addBands(ee.Image([cld_prb, is_cloud]))


      def clip2lakes(image):
        '''Clips an image based on the lake boundaries'''
        return image.clipToCollection(lakes)

      # Get percentile cover - saved as metadata


      # Mosaic images by date, orbit, - basically combines images together that were taken on the same day
      def mosaicBy(imcol):
        '''Takes an image collection (imcol) and creates a mosaic for each day
        Returns: An image collection of daily mosaics'''
        #return the collection as a list of images (not an image collection)
        imlist = imcol.toList(imcol.size())
        # Get all the dates as list
        def imdate(im):
          date = ee.Image(im).date().format("YYYY-MM-dd")
          return date
        all_dates = imlist.map(imdate)
        # get all orbits as list
        def orbitId(im):
          orb = ee.Image(im).get('SENSING_ORBIT_NUMBER')
          return orb
        all_orbits = imlist.map(orbitId)
        # get all spacecraft names as list
        def spacecraft(im):
          return ee.Image(im).get('SPACECRAFT_NAME')
        all_spNames = imlist.map(spacecraft)
        # this puts dates, orbits and names into a nested list
        concat_all = all_dates.zip(all_orbits).zip(all_spNames);
        # here we unnest the list with flatten, and then concatenate the list elements with " "
        def concat(el):
          return ee.List(el).flatten().join(" ")
        concat_all = concat_all.map(concat)
        # here, just get distinct combintations of date, orbit and name
        concat_unique = concat_all.distinct()
        # mosaic
        def mosaicIms(d):
          d1 = ee.String(d).split(" ")
          date1 = ee.Date(d1.get(0))
          orbit = ee.Number.parse(d1.get(1)).toInt()
          spName = ee.String(d1.get(2))
          im = imcol.filterDate(date1, date1.advance(1, "day")).filterMetadata('SPACECRAFT_NAME', 'equals', spName).filterMetadata('SENSING_ORBIT_NUMBER','equals', orbit).mosaic()
          return im.set(
              "system:time_start", date1.millis(),
              "system:date", date1.format("YYYY-MM-dd"),
              "system:id", d1)
        mosaic_imlist = concat_unique.map(mosaicIms)
        return ee.ImageCollection(mosaic_imlist)

      # Apply cloud mask to other bands
      def applyMask(image):
        img = image.updateMask(image.select('clouds').neq(1))
        ice = image.select('B4').gte(1050)
        maskedice = img.mask(ice.Not())
        return maskedice

      ###########################################################################
      ## ***WATER CLASSIFICATION METHODS***
      ###############################################################################

      # Define NDWI image
      def ndwi(image):
        '''Adds an NDWI band to the input image'''
        return image.normalizedDifference(['B3', 'B8']).rename('NDWI').multiply(1000)

      # Basic ndwi classification
      def ndwi_classify(image):
        '''Creates a binary image based on an NDWI threshold of 0'''
        ndwimask = image.select('NDWI')
        water = ndwimask.gte(0)
        land = ndwimask.lt(0)
        return(water)

      # OTSU thresholding from histogram
      def otsu(histogram):
        '''Returns the NDWI threshold for binary water classification'''
        counts = ee.Array(ee.Dictionary(histogram).get('histogram'))
        means = ee.Array(ee.Dictionary(histogram).get('bucketMeans'))
        size = means.length().get([0])
        total = counts.reduce(ee.Reducer.sum(), [0]).get([0])
        sum = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0])
        mean = sum.divide(total)
        indices = ee.List.sequence(1, size)
        def func_xxx(i):
          '''Compute between sum of squares, where each mean partitions the data.'''
          aCounts = counts.slice(0, 0, i)
          aCount = aCounts.reduce(ee.Reducer.sum(), [0]).get([0])
          aMeans = means.slice(0, 0, i)
          aMean = aMeans.multiply(aCounts) \
              .reduce(ee.Reducer.sum(), [0]).get([0]) \
              .divide(aCount)
          bCount = total.subtract(aCount)
          bMean = sum.subtract(aCount.multiply(aMean)).divide(bCount)
          return aCount.multiply(aMean.subtract(mean).pow(2)).add(
                bCount.multiply(bMean.subtract(mean).pow(2)))
        bss = indices.map(func_xxx)
        # Return the mean value corresponding to the maximum BSS.
        return means.sort(bss).get([-1])

      # OTSU thresholding for an image
      def otsu_thresh(water_image):
        '''Calculate NDWI and create histogram. Return the OTSU threshold.'''
        NDWI = ndwi(water_image).select('NDWI').updateMask(water_image.select('clouds').neq(1))
        histogram = ee.Dictionary(NDWI.reduceRegion(
          geometry = roi,
          reducer = ee.Reducer.histogram(255, 2).combine('mean', None, True).combine('variance', None, True),
          scale = pixScale,
          maxPixels = 1e12
        ))
        return otsu(histogram.get('NDWI_histogram'))

      # Classify an image using OTSU threshold.
      def otsu_classify(water_image):
        '''(1) Calculate NDWI and create histogram. (2) Calculate NDWI threshold for
        binary classification using OTSU method. (3) Classify image and add layer to input image.
        '''
        NDWI = ndwi(water_image).select('NDWI')
        histogram = ee.Dictionary(NDWI.reduceRegion(
          geometry = roi,
          reducer = ee.Reducer.histogram(255, 2).combine('mean', None, True).combine('variance', None, True),
          scale = pixScale,
          maxPixels = 1e12
        ))
        threshold = otsu(histogram.get('NDWI_histogram'))
        otsu_classed = NDWI.gt(ee.Number(threshold)).And(water_image.select('B8').lt(2000)).rename('otsu_classed')
        return water_image.addBands([otsu_classed])

      def adaptive_thresholding(water_image):
        '''Takes an image clipped to lakes and returns the water mask'''
        NDWI = ndwi(water_image).select('NDWI')
        threshold = ee.Number(otsu_thresh(water_image))
        threshold = threshold.divide(10).round().multiply(10)
        # get fixed histogram
        histo = NDWI.reduceRegion(
            geometry = roi,
            reducer = ee.Reducer.fixedHistogram(-1000, 1000, 200),
            scale = pixScale, # This was 30, keep at 10!?!?
            maxPixels = 1e12
        )
        hist = ee.Array(histo.get('NDWI'))
        counts = hist.cut([-1,1])
        buckets = hist.cut([-1,0])
        #find split points from otsu threshold
        threshold = ee.Array([threshold]).toList()
        buckets_list = buckets.toList()
        split = buckets_list.indexOf(threshold)
        # split into land and water slices
        land_slice = counts.slice(0,0,split)
        water_slice = counts.slice(0,split.add(1),-1)
        # find max of land and water slices
        land_max = land_slice.reduce(ee.Reducer.max(),[0])
        water_max = water_slice.reduce(ee.Reducer.max(),[0])
        land_max = land_max.toList().get(0)
        water_max = water_max.toList().get(0)
        land_max = ee.List(land_max).getNumber(0)
        water_max = ee.List(water_max).getNumber(0)
        #find difference between land, water and otsu val
        counts_list = counts.toList()
        otsu_val = ee.Number(counts_list.get(split))
        otsu_val = ee.List(otsu_val).getNumber(0)
        land_prom = ee.Number(land_max).subtract(otsu_val)
        water_prom = ee.Number(water_max).subtract(otsu_val)
        #find land and water buckets corresponding to 0.9 times the prominence
        land_thresh = ee.Number(land_max).subtract((land_prom).multiply(ee.Number(0.9)))
        water_thresh = ee.Number(water_max).subtract((water_prom).multiply(ee.Number(0.9)))
        land_max_ind = land_slice.argmax().get(0)
        water_max_ind = water_slice.argmax().get(0)
        li = ee.Number(land_max_ind).subtract(1)
        li = li.max(ee.Number(1))
        wi = ee.Number(water_max_ind).add(1)
        wi = wi.min(ee.Number(199))
        land_slice2 = land_slice.slice(0,li,-1).subtract(land_thresh)
        water_slice2 = water_slice.slice(0,0,wi).subtract(water_thresh)
        land_slice2  = land_slice2.abs().multiply(-1)
        water_slice2 = water_slice2.abs().multiply(-1)
        land_index = ee.Number(land_slice2.argmax().get(0)).add(land_max_ind)
        water_index = ee.Number(water_slice2.argmax().get(0)).add(split)
        land_level = ee.Number(buckets_list.get(land_index))
        water_level = ee.Number(buckets_list.get(water_index))
        land_level = ee.Number(ee.List(land_level).get(0)).add(5)
        water_level = ee.Number(ee.List(water_level).get(0)).add(5)
        #calculate water fraction and classify
        water_fraction = (NDWI.subtract(land_level)).divide(water_level.subtract(land_level)).multiply(100).rename('water_fraction')
        #water_fraction = conditional(water_fraction) #sets values less than 0 to 0 and greater than 100 to 100
        water_75 = water_fraction.gte(75).rename('water_75'); #note, this is a non-binary classification, so we use 75% water as "water"
        all_mask = water_image.select('B2').gt(5).rename('all_mask')
        cloud_mask_ed = water_image.select('clouds').neq(0).rename('cloud_mask_ed')
        return water_image.addBands([water_fraction,water_75,NDWI,cloud_mask_ed])

      def binaryImage(image):
        '''takes a multiband image and returns just the binary water_75 band'''
        img = image.select('water_75').rename('water_occurance')
        return img
      def waterImage(image):
        '''takes a multiband image and returns just the water fraction band'''
        img = image.select('water_fraction')
        return img



      ## ***Lake Occurrence Raster***
      # (1) image pre-process:

      # Get images and cloud probability, add cloud bands
      images = get_s2_sr_cld_col(roi).map(add_cloud_bands)
     # if images.size().getInfo() == 0:
      #  print('No S2 images for ' + description)
       # continue
     # print(images.size().getInfo())
      # Mosaic images and clip to ROI
      # Clip image
      def clip_image(image):
          return image.clip(roi)

      images_all = mosaicBy(images).map(clip_image)
      # Get percent cover for each mosaic
      image_mask = images_all.select('B2').mean().clip(roi).gte(0)  #First, calculate total number of pixels
      totPixels = ee.Number(image_mask.reduceRegion(
          reducer = ee.Reducer.count(),
          scale = 100,
          geometry = roi,
          maxPixels = 1e12
          ).values().get(0))

      def getCover(image):
        actPixels = ee.Number(image.updateMask(image.select('clouds').neq(1)).reduceRegion(
                      reducer = ee.Reducer.count(),
                      scale = 100,
                      geometry = roi,
                      maxPixels=1e12,
                      ).values().get(0))
        percCover = actPixels.divide(totPixels).multiply(100).round()
        ice = image.select('B4').gte(1050)
        maskedice = image.mask(ice.Not())
        icePixels = ee.Number(maskedice
                .reduceRegion(
                      reducer = ee.Reducer.count(),
                      scale = 100,
                      geometry = roi,
                      maxPixels = 1e12,
                      ).values().get(0))
        percIce = icePixels.divide(totPixels).multiply(100).round()
      # number as output
        return image.set('cloud_free_cover', percCover,'actPixels',actPixels, 'ice_free_cover', percIce)

      coverimages = images_all.map(getCover)
      # Filter by percent cover, remove images with the mask covering more than 30% of the ROI)
      cleancoverimages = coverimages.filterMetadata('cloud_free_cover','greater_than',70).filterMetadata('ice_free_cover','greater_than',10)

     # if cleancoverimages.size().getInfo() == 0:
      #  print("No images meet the cloud and ice cover criteria for " + description)
       # continue

      # Apply cloud mask to other bands
      cloudmaskedimgs = cleancoverimages.map(applyMask)
      # (2) water classification:

      # adaptive thresholding on every lake image
      thesholdedimages = cloudmaskedimgs.map(adaptive_thresholding)
      # create image collection of just binary water images
      waterimages = thesholdedimages.map(binaryImage)

      #clip to just include lakes
      lakeimages = waterimages.map(clip2lakes)#.filter(ee.Filter.listContains("system:band_names", "water_occurance"))
      # get max of images
      finalimage = lakeimages.max()

      task = ee.batch.Export.image.toAsset(**{
              'image': ee.Image(finalimage),
              'description': description,
              'assetId': 'projects/alpod-412314/assets/' + regionfolder  + description,
              'scale': 10,
              'region': roi,
              'maxPixels': 1e12,
              })
      task.start()
      print("Image export " +  description +  " has started")
     # else:
      #  print("No images for " + description)


Image export 05WNU_2016_17 has started
Image export 05WNU_2016_18 has started
Image export 05WNU_2016_19 has started
Image export 05WNU_2016_20 has started
Image export 05WNU_2016_21 has started
Image export 05WNU_2016_22 has started
Image export 05WNU_2016_23 has started
Image export 05WNU_2016_24 has started
Image export 05WNU_2016_25 has started
Image export 05WNU_2016_26 has started
Image export 05WNU_2016_27 has started
Image export 05WNU_2016_28 has started
Image export 05WNU_2016_29 has started
Image export 05WNU_2016_30 has started
Image export 05WNU_2016_31 has started
Image export 05WNU_2016_32 has started
Image export 05WNU_2016_33 has started
Image export 05WNU_2016_34 has started
Image export 05WNU_2016_35 has started
Image export 05WNU_2016_36 has started
Image export 05WNU_2016_37 has started
Image export 05WNU_2016_38 has started
Image export 05WNU_2016_39 has started
